In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %cd /content/drive/MyDrive/

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11135164849872256422
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1745276110
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 842067009866871888
 physical_device_desc: "device: 0, name: NVIDIA RTX A2000 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [4]:
import pandas as pd
#import pandas_profiling
import numpy as np
import cupy as cp
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import roc_auc_score
from sklearn.tree import plot_tree
from sklearn.metrics import mean_squared_error

from six import StringIO

from IPython.display import Image
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import math
from scipy import stats
import numpy as np
import optuna.integration.lightgbm as lgb
import pandas as pd
import tensorflow as tf

from datetime import datetime
from tensorflow import feature_column
from tensorflow.keras import layers

from xgboost.sklearn import XGBClassifier


print("OK\n")

OK



In [5]:
app_train_origin = pd.read_csv('./home-credit-default-risk/exports/app_train.csv')
app_test_origin =  pd.read_csv('./home-credit-default-risk/exports/app_test.csv')
#bureau_origin = pd.read_csv('./home-credit-default-risk/bureau.csv')
#bureau_balance_origin = pd.read_csv('./home-credit-default-risk/bureau_balance.csv')
pos_cash_balance_origin = pd.read_csv('./home-credit-default-risk/POS_CASH_balance.csv')
#credit_card_balance_origin = pd.read_csv('./home-credit-default-risk/credit_card_balance.csv')
previous_application_origin = pd.read_csv('./home-credit-default-risk/previous_application.csv')
installments_payments_origin = pd.read_csv('./home-credit-default-risk/installments_payments.csv')
# custo data
app_train_hc_only_revo_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_revo.csv')
app_test_hc_only_revo_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_revo.csv')
app_train_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_test_1000_mean_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_revo_installment_payment_ratio_1000_mean_mean.csv')
app_train_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/train_hc_only_revo_annuity_to_max_installment_ratio.csv')
app_test_max_installment_ratio_origin = pd.read_csv('./home-credit-default-risk/exports/test_hc_only_revo_annuity_to_max_installment_ratio.csv')
app_train_origin.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_16,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_YEAR,CREDIT_TERM,DAYS_EMPLOYED_PERCENT,credit_annuity_ratio,DAYS_EMPLOYED_ANOM,credit_goods_price_ratio,credit_downpayment,AGE_INT
0,100002,1,0,1,0,0,202500.0,406597.5,24700.5,351000.0,...,0,0.0,1.0,0.060749,-0.067329,16.461104,False,1.158397,-55597.5,-25.920548
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,0,0.0,0.0,0.027598,-0.070862,36.234085,False,1.145199,-164002.5,-45.931507
2,100004,0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0.0,0.0,0.050000,-0.011814,20.000000,False,1.000000,0.0,-52.180822
3,100006,0,0,0,0,0,135000.0,312682.5,29686.5,297000.0,...,0,NaN,NaN,0.094941,-0.159905,10.532818,False,1.052803,-15682.5,-52.068493
4,100007,0,0,1,0,0,121500.0,513000.0,21865.5,513000.0,...,0,0.0,0.0,0.042623,-0.152418,23.461618,False,1.000000,0.0,-54.608219


In [6]:
app_train = app_train_origin.copy()
app_test = app_test_origin.copy()
pos_cash_balance = pos_cash_balance_origin.copy()
previous_application = previous_application_origin.copy()
installments_payments = installments_payments_origin.copy()
app_train_revo = app_train_hc_only_revo_origin.copy()
app_test_revo = app_test_hc_only_revo_origin.copy()
app_train_1000_mean = app_train_1000_mean_origin.copy()
app_test_1000_mean = app_test_1000_mean_origin.copy()
app_train_max_installment_ratio = app_train_max_installment_ratio_origin.copy()
app_test_max_installment_ratio = app_test_max_installment_ratio_origin.copy()

### applicationの複製

In [7]:
#app_train = app_train[['SK_ID_CURR','TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
#app_test = app_test[['SK_ID_CURR','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']].copy()
app_train = app_train[['SK_ID_CURR','TARGET']].copy()
app_test = app_test[['SK_ID_CURR']].copy()
len(app_test)

48744

In [8]:
app_train.head()

,SK_ID_CURR,TARGET
0,100002,1
1,100003,0
2,100004,0
3,100006,0
4,100007,0


### 申し込み月におけるCssh loanの情報をマージ

In [9]:
app_train_revo = app_train_revo.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_revo = app_test_revo.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
len(app_train_revo)

35586

In [10]:
# trainへマージ
app_train = pd.merge(app_train, app_train_revo, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_revo, on='SK_ID_CURR', how='left')
len(app_train)

307511

### installment_payment_ratio_1000_mean_meanをマージ

In [11]:
app_train_1000_mean = app_train_1000_mean.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_1000_mean = app_test_1000_mean.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_train_1000_mean.head()

,SK_ID_CURR,revo_installment_payment_ratio_1000_mean_mean
0,100011,0.000000
1,100043,-1260.462857
2,100048,0.000000
3,100049,-289.125000
4,100075,0.000000


In [12]:
#app_train_1000_mean.drop(['sk_id_prev'], axis=1, inplace=True)
#app_test_1000_mean.drop(['sk_id_prev'], axis=1, inplace=True)
#app_train_1000_mean.head()

In [13]:
# trainへマージ
app_train = pd.merge(app_train, app_train_1000_mean, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_1000_mean, on='SK_ID_CURR', how='left')
len(app_train)

307511

In [14]:
app_train['revo_installment_payment_ratio_1000_mean_mean']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
307506   NaN
307507   NaN
307508   NaN
307509   NaN
307510   NaN
Name: revo_installment_payment_ratio_1000_mean_mean, Length: 307511, dtype: float64

In [15]:
app_train['revo_installment_payment_ratio_1000_mean_mean'].isnull().sum()

255531

In [16]:
#app_train['amt_annuity'].fillna(0, inplace=True)
#app_test['amt_annuity'].fillna(0, inplace=True)

In [17]:
app_train['revo_amt_annuity'].isnull().sum()

271925

### annuity_to_max_installment_ratioをマージ

In [18]:
app_train_max_installment_ratio = app_train_max_installment_ratio.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_max_installment_ratio = app_test_max_installment_ratio.rename(columns={'sk_id_curr': 'SK_ID_CURR'})
app_test_max_installment_ratio.head()

,SK_ID_CURR,revo_annuity_to_max_installment_ratio
0,100028,-1815.4815
1,100042,-2035.2941
2,100067,-593.0000
3,100107,-5422.5000
4,100330,-3950.2857


In [19]:
# trainへマージ
app_train = pd.merge(app_train, app_train_max_installment_ratio, on='SK_ID_CURR', how='left')
# testへマージ
app_test = pd.merge(app_test, app_test_max_installment_ratio, on='SK_ID_CURR', how='left')
len(app_train)

307511

### AMT_ANNUITY(ローン支払い額)

In [20]:
max(app_train['revo_amt_annuity'])

nan

In [21]:
min(app_train['revo_amt_annuity'])

nan

In [22]:
app_train[['revo_amt_annuity', 'TARGET']].corr()

,revo_amt_annuity,TARGET
revo_amt_annuity,1.000000,-0.023584
TARGET,-0.023584,1.000000


### AMT_APPLICATION|借り入れ申し込み金額|

In [23]:
#app_train['amt_application'].fillna(0, inplace=True)
#app_test['amt_application'].fillna(0, inplace=True)

In [24]:
app_train[['revo_amt_application', 'TARGET']].corr()

,revo_amt_application,TARGET
revo_amt_application,1.000000,0.021231
TARGET,0.021231,1.000000


### AMT_CREDIT(借り入れ承認金額(最終的な貸付金額))

In [25]:
max(app_train['revo_amt_credit'])

nan

In [26]:
min(app_train['revo_amt_credit'])

nan

In [27]:
app_train[['revo_amt_credit', 'TARGET']].corr()

,revo_amt_credit,TARGET
revo_amt_credit,1.000000,-0.022719
TARGET,-0.022719,1.000000


### AMT_DOWN_PAYMENT(頭金) Cash loans の場合は不要?

In [28]:
app_train[['revo_amt_down_payment', 'TARGET']].corr()

,revo_amt_down_payment,TARGET
revo_amt_down_payment,1.000000,-0.010726
TARGET,-0.010726,1.000000


In [29]:
#app_train.drop(['revo_amt_down_payment'], axis=1, inplace=True)
#app_test.drop(['revo_amt_down_payment'], axis=1, inplace=True)

### AMT_GOODS_PRICE(クライアントが要求した商品価格)

In [30]:
app_train[['revo_amt_goods_price', 'TARGET']].corr()

,revo_amt_goods_price,TARGET
revo_amt_goods_price,1.000000,0.021231
TARGET,0.021231,1.000000


### WEEKDAY_APPR_PROCESS_START(申請した曜日)

In [31]:
app_train['revo_weekday_appr_process_start'].replace(['TUESDAY', 'MONDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY'], [1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['revo_weekday_appr_process_start'].replace(['TUESDAY', 'MONDAY', 'WEDNESDAY', 'THURSDAY', 'FRIDAY', 'SATURDAY', 'SUNDAY'], [1, 2, 3, 4, 5, 6, 7], inplace=True)

In [32]:
app_train['revo_weekday_appr_process_start'].value_counts()

3.0    5908
1.0    5878
5.0    5580
4.0    5549
2.0    5438
6.0    4580
7.0    2653
Name: revo_weekday_appr_process_start, dtype: int64

In [33]:
app_train[['revo_weekday_appr_process_start', 'TARGET']].corr()

,revo_weekday_appr_process_start,TARGET
revo_weekday_appr_process_start,1.000000,0.003813
TARGET,0.003813,1.000000


In [34]:
app_train.drop(['revo_weekday_appr_process_start'], axis=1, inplace=True)
app_test.drop(['revo_weekday_appr_process_start'], axis=1, inplace=True)

### HOUR_APPR_PROCESS_START(クライアントが応募した時間)

In [35]:
app_train['revo_hour_appr_process_start'].value_counts()

11.0    4079
10.0    3957
12.0    3787
13.0    3650
14.0    3259
15.0    2976
9.0     2843
16.0    2556
17.0    1990
8.0     1756
18.0    1318
7.0     1033
19.0     674
6.0      564
5.0      313
20.0     274
4.0      193
21.0     123
3.0      108
22.0      45
2.0       38
23.0      26
0.0       13
1.0       11
Name: revo_hour_appr_process_start, dtype: int64

In [36]:
app_train[['revo_hour_appr_process_start', 'TARGET']].corr()

,revo_hour_appr_process_start,TARGET
revo_hour_appr_process_start,1.000000,-0.030777
TARGET,-0.030777,1.000000


### FLAG_LAST_APPL_PER_CONTRACT|最も最新の過去のローン申請だった場合のフラグ(店員による間違いがあるかもしれんけど)|

In [37]:
app_train['revo_flag_last_appl_per_contract'].value_counts()

Y    35586
Name: revo_flag_last_appl_per_contract, dtype: int64

In [38]:
app_train.drop(['revo_flag_last_appl_per_contract'], axis=1, inplace=True)
app_test.drop(['revo_flag_last_appl_per_contract'], axis=1, inplace=True)

### NFLAG_LAST_APPL_IN_DAY(1日の中での最後の申し込みだった場合のフラグ)

In [39]:
app_train['revo_nflag_last_appl_in_day'].value_counts()

1.0    35561
0.0       25
Name: revo_nflag_last_appl_in_day, dtype: int64

In [40]:
app_train[['revo_nflag_last_appl_in_day', 'TARGET']].corr()

,revo_nflag_last_appl_in_day,TARGET
revo_nflag_last_appl_in_day,1.000000,0.005826
TARGET,0.005826,1.000000


In [41]:
app_train.drop(['revo_nflag_last_appl_in_day'], axis=1, inplace=True)
app_test.drop(['revo_nflag_last_appl_in_day'], axis=1, inplace=True)

### RATE_DOWN_PAYMENT(前払い率)

In [42]:
app_train['revo_rate_down_payment'].value_counts()

0.000000    35332
0.217818        6
0.363030        5
0.155584        5
0.401244        3
            ...  
0.106220        1
0.107320        1
0.078177        1
0.104541        1
0.269918        1
Name: revo_rate_down_payment, Length: 208, dtype: int64

In [43]:
app_train[['revo_rate_down_payment', 'TARGET']].corr()

,revo_rate_down_payment,TARGET
revo_rate_down_payment,1.000000,-0.010131
TARGET,-0.010131,1.000000


### RATE_INTEREST_PRIMARY(金利)

In [44]:
app_train['revo_rate_down_payment'].value_counts()

0.000000    35332
0.217818        6
0.363030        5
0.155584        5
0.401244        3
            ...  
0.106220        1
0.107320        1
0.078177        1
0.104541        1
0.269918        1
Name: revo_rate_down_payment, Length: 208, dtype: int64

In [45]:
app_train[['revo_rate_down_payment', 'TARGET']].corr()

,revo_rate_down_payment,TARGET
revo_rate_down_payment,1.000000,-0.010131
TARGET,-0.010131,1.000000


### RATE_INTEREST_PRIVILEGED(金利)

In [46]:
app_train['revo_rate_interest_privileged'].value_counts()

0.0    35586
Name: revo_rate_interest_privileged, dtype: int64

In [47]:
app_train[['revo_rate_interest_privileged', 'TARGET']].corr()

,revo_rate_interest_privileged,TARGET
revo_rate_interest_privileged,NaN,NaN
TARGET,NaN,1.0


In [48]:
app_train.drop(['revo_rate_interest_privileged'], axis=1, inplace=True)
app_test.drop(['revo_rate_interest_privileged'], axis=1, inplace=True)

### NAME_CASH_LOAN_PURPOSE(現金融資の目的)

In [49]:
app_train['revo_name_cash_loan_purpose'].value_counts()

XAP    35586
Name: revo_name_cash_loan_purpose, dtype: int64

In [50]:
app_train.drop(['revo_name_cash_loan_purpose'], axis=1, inplace=True)
app_test.drop(['revo_name_cash_loan_purpose'], axis=1, inplace=True)

### NAME_CONTRACT_STATUS(契約状況)

In [51]:
app_train['revo_name_contract_status'].value_counts()

Approved    35586
Name: revo_name_contract_status, dtype: int64

In [52]:
app_train.drop(['revo_name_contract_status'], axis=1, inplace=True)
app_test.drop(['revo_name_contract_status'], axis=1, inplace=True)

### DAYS_DECISION(今回申込日との日数差)

In [53]:
app_train['revo_days_decision'].value_counts()

-245.0     83
-252.0     72
-215.0     71
-269.0     71
-244.0     71
           ..
-2130.0     1
-1881.0     1
-1907.0     1
-1979.0     1
-1499.0     1
Name: revo_days_decision, Length: 2623, dtype: int64

In [54]:
app_train[['revo_days_decision', 'TARGET']].corr()

,revo_days_decision,TARGET
revo_days_decision,1.000000,0.099104
TARGET,0.099104,1.000000


### NAME_PAYMENT_TYPE(支払い方法)

In [55]:
app_train['revo_name_payment_type'].value_counts()

XNA    35586
Name: revo_name_payment_type, dtype: int64

In [56]:
app_train.drop(['revo_name_payment_type'], axis=1, inplace=True)
app_test.drop(['revo_name_payment_type'], axis=1, inplace=True)

### CODE_REJECT_REASON(審査落ち理由)

In [57]:
app_train['revo_code_reject_reason'].value_counts()

XAP    35586
Name: revo_code_reject_reason, dtype: int64

In [58]:
app_train.drop(['revo_code_reject_reason'], axis=1, inplace=True)
app_test.drop(['revo_code_reject_reason'], axis=1, inplace=True)

### NAME_TYPE_SUITE(申請時同行者)

In [59]:
app_train['revo_name_type_suite'].value_counts()

Unaccompanied      8366
Family             1559
Spouse, partner     387
Children            122
Other_B              89
Other_A              51
Group of people      11
Name: revo_name_type_suite, dtype: int64

In [60]:
columns = ['Unaccompanied', 'Family', 'Spouse, partner', 'Children', 'Other_B', 'Other_A', 'Group of people']
app_train['revo_name_type_suite'].replace(columns, [1, 2, 3, 4, 5, 6, 7], inplace=True)
app_test['revo_name_type_suite'].replace(columns, [1, 2, 3, 4, 5, 6, 7], inplace=True)

In [61]:
app_train['revo_name_type_suite'].value_counts()

1.0    8366
2.0    1559
3.0     387
4.0     122
5.0      89
6.0      51
7.0      11
Name: revo_name_type_suite, dtype: int64

In [62]:
app_train[['revo_name_type_suite', 'TARGET']].corr()

,revo_name_type_suite,TARGET
revo_name_type_suite,1.000000,0.010551
TARGET,0.010551,1.000000


### NAME_CLIENT_TYPE(新規顧客などの顧客タイプ)

In [63]:
app_train['revo_name_client_type'].value_counts()

Repeater     30533
New           2633
Refreshed     2416
XNA              4
Name: revo_name_client_type, dtype: int64

In [64]:
columns = ['Repeater', 'Refreshed', 'New', 'XNA']
app_train['revo_name_client_type'].replace(columns, [1, 2, 3, 4], inplace=True)
app_test['revo_name_client_type'].replace(columns, [1, 2, 3, 4], inplace=True)

In [65]:
app_train['revo_name_client_type'].value_counts()

1.0    30533
3.0     2633
2.0     2416
4.0        4
Name: revo_name_client_type, dtype: int64

In [66]:
app_train[['revo_name_client_type', 'TARGET']].corr()

,revo_name_client_type,TARGET
revo_name_client_type,1.000000,0.054188
TARGET,0.054188,1.000000


### NAME_GOODS_CATEGORY(申請した商品の種類)

In [67]:
app_train['revo_name_goods_category'].value_counts()

XNA                         35332
Consumer Electronics          181
Audio/Video                    43
Mobile                         14
Computers                      10
Furniture                       5
Photo / Cinema Equipment        1
Name: revo_name_goods_category, dtype: int64

In [68]:
replace_columns = app_train['revo_name_goods_category'].unique().tolist()
replace_columns = ['xxxx' if e == "NaN" or type(e) is float and math.isnan(e) else e for e in replace_columns]
###replace_columns.remove('xxxx')
replace_value = [x for x in range(len(replace_columns))]
app_train['revo_name_goods_category'].replace(replace_columns, replace_value, inplace=True)
app_test['revo_name_goods_category'].replace(replace_columns, replace_value, inplace=True)
app_train['revo_name_goods_category'].value_counts()

1.0    35332
2.0      181
5.0       43
3.0       14
4.0       10
6.0        5
7.0        1
Name: revo_name_goods_category, dtype: int64

In [69]:
app_train['revo_name_goods_category'].value_counts()

1.0    35332
2.0      181
5.0       43
3.0       14
4.0       10
6.0        5
7.0        1
Name: revo_name_goods_category, dtype: int64

In [70]:
app_train[['revo_name_goods_category', 'TARGET']].corr()

,revo_name_goods_category,TARGET
revo_name_goods_category,1.000000,-0.008904
TARGET,-0.008904,1.000000


In [71]:
app_train.drop(['revo_name_goods_category'], axis=1, inplace=True)
app_test.drop(['revo_name_goods_category'], axis=1, inplace=True)

### name_portfolio

In [72]:
app_train['revo_name_portfolio'].value_counts()

Cards    35586
Name: revo_name_portfolio, dtype: int64

In [73]:
app_train.drop(['revo_name_portfolio'], axis=1, inplace=True)
app_test.drop(['revo_name_portfolio'], axis=1, inplace=True)

### name_product_type

In [74]:
app_train['revo_name_product_type'].value_counts()

x-sell     27140
walk-in     8446
Name: revo_name_product_type, dtype: int64

In [75]:
replace_columns = app_train['revo_name_product_type'].unique().tolist()
replace_columns = ['xxxx' if e == "NaN" or type(e) is float and math.isnan(e) else e for e in replace_columns]
replace_columns.remove('xxxx')
replace_value = [x for x in range(len(replace_columns))]
app_train['revo_name_product_type'].replace(replace_columns, replace_value, inplace=True)
app_test['revo_name_product_type'].replace(replace_columns, replace_value, inplace=True)
app_train['revo_name_product_type'].value_counts()

0.0    27140
1.0     8446
Name: revo_name_product_type, dtype: int64

In [76]:
app_train[['revo_name_product_type', 'TARGET']].corr()

,revo_name_product_type,TARGET
revo_name_product_type,1.00000,0.07536
TARGET,0.07536,1.00000


### channel_type

In [77]:
app_train['revo_channel_type'].value_counts()

Credit and cash offices       17751
Country-wide                   7130
Contact center                 4845
AP+ (Cash loan)                2812
Stone                          1664
Regional / Local               1169
Channel of corporate sales      214
Car dealer                        1
Name: revo_channel_type, dtype: int64

In [78]:
replace_columns = app_train['revo_channel_type'].unique().tolist()
replace_columns = ['xxxx' if e == "NaN" or type(e) is float and math.isnan(e) else e for e in replace_columns]
replace_columns.remove('xxxx')
replace_value = [x for x in range(len(replace_columns))]
app_train['revo_channel_type'].replace(replace_columns, replace_value, inplace=True)
app_test['revo_channel_type'].replace(replace_columns, replace_value, inplace=True)
app_train['revo_channel_type'].value_counts()

1.0    17751
0.0     7130
2.0     4845
3.0     2812
4.0     1664
6.0     1169
5.0      214
7.0        1
Name: revo_channel_type, dtype: int64

In [79]:
app_train[['revo_channel_type', 'TARGET']].corr()

,revo_channel_type,TARGET
revo_channel_type,1.00000,0.00912
TARGET,0.00912,1.00000


In [80]:
app_train.drop(['revo_channel_type'], axis=1, inplace=True)
app_test.drop(['revo_channel_type'], axis=1, inplace=True)

### sellerplace_area

In [81]:
app_train['revo_sellerplace_area'].value_counts()

-1.0       20904
 0.0        2284
 4.0         622
 5.0         455
 50.0        394
           ...  
 1331.0        1
 268.0         1
 1304.0        1
 621.0         1
 510.0         1
Name: revo_sellerplace_area, Length: 927, dtype: int64

In [82]:
app_train[['revo_sellerplace_area', 'TARGET']].corr()

,revo_sellerplace_area,TARGET
revo_sellerplace_area,1.000000,-0.006568
TARGET,-0.006568,1.000000


In [83]:
app_train.drop(['revo_sellerplace_area'], axis=1, inplace=True)
app_test.drop(['revo_sellerplace_area'], axis=1, inplace=True)

### name_seller_industry

In [84]:
app_train['revo_name_seller_industry'].value_counts()

XNA                     25610
Consumer electronics     5820
Connectivity             3733
Furniture                 267
Clothing                   63
Construction               54
Industry                   19
Auto technology            11
MLM partners                6
Jewelry                     3
Name: revo_name_seller_industry, dtype: int64

In [85]:
replace_columns = app_train['revo_name_seller_industry'].unique().tolist()
replace_columns = ['xxxx' if e == "NaN" or type(e) is float and math.isnan(e) else e for e in replace_columns]
replace_columns.remove('xxxx')
replace_value = [x for x in range(len(replace_columns))]
app_train['revo_name_seller_industry'].replace(replace_columns, replace_value, inplace=True)
app_test['revo_name_seller_industry'].replace(replace_columns, replace_value, inplace=True)
app_train['revo_name_seller_industry'].value_counts()

1.0    25610
2.0     5820
0.0     3733
3.0      267
8.0       63
7.0       54
5.0       19
4.0       11
9.0        6
6.0        3
Name: revo_name_seller_industry, dtype: int64

In [86]:
app_train[['revo_name_seller_industry', 'TARGET']].corr()

,revo_name_seller_industry,TARGET
revo_name_seller_industry,1.000000,-0.006876
TARGET,-0.006876,1.000000


In [87]:
app_train.drop(['revo_name_seller_industry'], axis=1, inplace=True)
app_test.drop(['revo_name_seller_industry'], axis=1, inplace=True)

### cnt_payment

In [88]:
app_train['revo_cnt_payment'].value_counts()

0.0    35586
Name: revo_cnt_payment, dtype: int64

In [89]:
app_train[['revo_cnt_payment', 'TARGET']].corr()

,revo_cnt_payment,TARGET
revo_cnt_payment,NaN,NaN
TARGET,NaN,1.0


In [90]:
app_train.drop(['revo_cnt_payment'], axis=1, inplace=True)
app_test.drop(['revo_cnt_payment'], axis=1, inplace=True)

### name_yield_group

In [91]:
app_train['revo_name_yield_group'].value_counts()

XNA    35586
Name: revo_name_yield_group, dtype: int64

In [92]:
app_train.drop(['revo_name_yield_group'], axis=1, inplace=True)
app_test.drop(['revo_name_yield_group'], axis=1, inplace=True)

### product_combination

In [93]:
app_train['revo_product_combination'].value_counts()

Card X-Sell    20791
Card Street    14795
Name: revo_product_combination, dtype: int64

In [94]:
replace_columns = app_train['revo_product_combination'].unique().tolist()
replace_columns = ['xxxx' if e == "NaN" or type(e) is float and math.isnan(e) else e for e in replace_columns]
replace_columns.remove('xxxx')
replace_value = [x for x in range(len(replace_columns))]
app_train['revo_product_combination'].replace(replace_columns, replace_value, inplace=True)
app_test['revo_product_combination'].replace(replace_columns, replace_value, inplace=True)
app_train['revo_product_combination'].value_counts()

0.0    20791
1.0    14795
Name: revo_product_combination, dtype: int64

In [95]:
app_train['revo_product_combination'].value_counts()

0.0    20791
1.0    14795
Name: revo_product_combination, dtype: int64

In [96]:
app_train[['revo_product_combination', 'TARGET']].corr()

,revo_product_combination,TARGET
revo_product_combination,1.000000,0.015433
TARGET,0.015433,1.000000


### days_first_drawing

In [97]:
app_train['revo_days_first_drawing'].value_counts()

 365243.0    115
-212.0        82
-210.0        79
-215.0        78
-249.0        77
            ... 
-1739.0        1
-1532.0        1
-1419.0        1
-1840.0        1
-1328.0        1
Name: revo_days_first_drawing, Length: 2664, dtype: int64

In [98]:
app_train[['revo_days_first_drawing', 'TARGET']].corr()

,revo_days_first_drawing,TARGET
revo_days_first_drawing,1.000000,-0.007476
TARGET,-0.007476,1.000000


In [99]:
app_train.drop(['revo_days_first_drawing'], axis=1, inplace=True)
app_test.drop(['revo_days_first_drawing'], axis=1, inplace=True)

### days_first_due

In [100]:
app_train['revo_days_first_due'].value_counts()

 365243.0    758
-208.0        95
-199.0        94
-167.0        89
-178.0        85
            ... 
-2044.0        1
-1684.0        1
-1773.0        1
-1645.0        1
-1592.0        1
Name: revo_days_first_due, Length: 2622, dtype: int64

In [101]:
app_train[['revo_days_first_due', 'TARGET']].corr()

,revo_days_first_due,TARGET
revo_days_first_due,1.000000,-0.011457
TARGET,-0.011457,1.000000


### days_last_due_1st_version

In [102]:
app_train['revo_days_last_due_1st_version'].value_counts()

365243.0    35576
Name: revo_days_last_due_1st_version, dtype: int64

In [103]:
app_train[['revo_days_last_due_1st_version', 'TARGET']].corr()

,revo_days_last_due_1st_version,TARGET
revo_days_last_due_1st_version,NaN,NaN
TARGET,NaN,1.0


In [104]:
app_train.drop(['revo_days_last_due_1st_version'], axis=1, inplace=True)
app_test.drop(['revo_days_last_due_1st_version'], axis=1, inplace=True)

### days_last_due

In [105]:
app_train['revo_days_last_due'].value_counts()

 365243.0    23868
-76.0           21
-107.0          18
-119.0          17
-297.0          17
             ...  
-2361.0          1
-1322.0          1
-1901.0          1
-1364.0          1
-1988.0          1
Name: revo_days_last_due, Length: 2482, dtype: int64

In [106]:
app_train[['revo_days_last_due', 'TARGET']].corr()

,revo_days_last_due,TARGET
revo_days_last_due,1.000000,0.084313
TARGET,0.084313,1.000000


### days_termination

In [107]:
app_train['revo_days_termination'].value_counts()

 365243.0    32998
-7.0            22
-99.0           21
-97.0           21
-113.0          21
             ...  
-788.0           1
-778.0           1
-943.0           1
-177.0           1
-870.0           1
Name: revo_days_termination, Length: 524, dtype: int64

In [108]:
app_train[['revo_days_termination', 'TARGET']].corr()

,revo_days_termination,TARGET
revo_days_termination,1.000000,0.024473
TARGET,0.024473,1.000000


### nflag_insured_on_approval

In [109]:
app_train['revo_nflag_insured_on_approval'].value_counts()

0.0    35185
1.0      391
Name: revo_nflag_insured_on_approval, dtype: int64

In [110]:
app_train[['revo_nflag_insured_on_approval', 'TARGET']].corr()

,revo_nflag_insured_on_approval,TARGET
revo_nflag_insured_on_approval,1.000000,-0.012439
TARGET,-0.012439,1.000000


### max_sk_dpd

In [111]:
app_train['revo_max_sk_dpd'].value_counts()

0.0      35096
1.0        102
8.0         73
5.0         69
7.0         38
         ...  
39.0         1
98.0         1
99.0         1
615.0        1
140.0        1
Name: revo_max_sk_dpd, Length: 117, dtype: int64

In [112]:
app_train[['revo_max_sk_dpd', 'TARGET']].corr()

,revo_max_sk_dpd,TARGET
revo_max_sk_dpd,1.000000,-0.000564
TARGET,-0.000564,1.000000


In [113]:
app_train.drop(['revo_max_sk_dpd'], axis=1, inplace=True)
app_test.drop(['revo_max_sk_dpd'], axis=1, inplace=True)

### installment_payment_ratio_1000_mean_mean

In [114]:
app_train['revo_installment_payment_ratio_1000_mean_mean'].value_counts()

 0.000000       37430
-86.538462         42
-70.312500         37
-75.000000         36
-80.357143         36
                ...  
-4743.404505        1
-24.196721          1
-350.312045         1
-76.658049          1
-80.110000          1
Name: revo_installment_payment_ratio_1000_mean_mean, Length: 12697, dtype: int64

In [115]:
app_train['revo_installment_payment_ratio_1000_mean_mean']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
          ..
307506   NaN
307507   NaN
307508   NaN
307509   NaN
307510   NaN
Name: revo_installment_payment_ratio_1000_mean_mean, Length: 307511, dtype: float64

In [116]:
app_train[['revo_installment_payment_ratio_1000_mean_mean', 'TARGET']].corr()

,revo_installment_payment_ratio_1000_mean_mean,TARGET
revo_installment_payment_ratio_1000_mean_mean,1.000000,-0.040611
TARGET,-0.040611,1.000000


### revo_annuity_to_max_installment_ratio

In [117]:
app_train['revo_annuity_to_max_installment_ratio'].value_counts()

-9114.7500    12
-6554.2500    11
-1567.0588    11
-3745.2857    11
-2804.5385    10
              ..
-2244.0000     1
-2041.0000     1
-158.4507      1
-3068.6667     1
-172.1627      1
Name: revo_annuity_to_max_installment_ratio, Length: 28471, dtype: int64

In [118]:
app_train[['revo_annuity_to_max_installment_ratio', 'TARGET']].corr()

,revo_annuity_to_max_installment_ratio,TARGET
revo_annuity_to_max_installment_ratio,1.00000,0.00516
TARGET,0.00516,1.00000


In [119]:
app_train.drop(['revo_annuity_to_max_installment_ratio'], axis=1, inplace=True)
app_test.drop(['revo_annuity_to_max_installment_ratio'], axis=1, inplace=True)

In [120]:
len(app_train)

307511

In [121]:
len(app_test)

48744

### CSVをEXPORT

In [122]:
### train
app_train.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_only_revo_train.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [123]:
### test
app_test.to_csv(
    path_or_buf="./home-credit-default-risk/exports/hc_only_revo_test.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [124]:
#num_cols = ['max_credit_day_overdue', 'max_days_credit_enddate', 'min_days_enddate_fact', 'max_amt_credit_max_overdue', 'cnt_credit_prolong_in_6month', 'sum_amt_credit_sum', 'sum_amt_credit_sum_debt', 'sum_amt_credit_sum_limit', 'sum_amt_credit_sum_overdue', 'avg_amt_annuity']

data = pd.concat([app_train, app_test], sort=False)
train = data[:len(app_train)]
test = data[len(app_test):]


y_train = train['TARGET']
X_train = app_train.copy().drop('TARGET', axis=1)
X_test = app_test.copy()

# 学習データに基づいてYao-Johnson変換を定義
#pt = PowerTransformer(method = 'yeo-johnson')
#pt.fit(X_train[num_cols])

# 変換後のデータで各列を置換
#X_train[num_cols] = pt.transform(X_train[num_cols])
#X_test[num_cols] = pt.transform(X_test[num_cols])

X_train.head()

,SK_ID_CURR,revo_amt_annuity,revo_amt_application,revo_amt_credit,revo_amt_down_payment,revo_amt_goods_price,revo_hour_appr_process_start,revo_rate_down_payment,revo_rate_interest_primary,revo_days_decision,revo_name_type_suite,revo_name_client_type,revo_name_product_type,revo_product_combination,revo_days_first_due,revo_days_last_due,revo_days_termination,revo_nflag_insured_on_approval,revo_installment_payment_ratio_1000_mean_mean
0,100002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
import xgboost as xgb
from sklearn.metrics import log_loss

y_preds = []
models = []
scores = []
oof_train = np.zeros((len(X_train),))
cv = KFold(n_splits=5, shuffle=True, random_state=0)

categorical_features = columns

params = {
    'booster': 'gbtree'
    ,'objective': 'binary:logistic'
    ,'silent': 1
    ,'random_state': 17
    ,'gpu_id': 0
    ,'tree_method': 'gpu_hist'
    ,'eval_metric': 'auc'
    ,'eta': 0.1
    #,'max_round': 100
    ,'max_depth': 8
    ,'subsample': 0.8
    ,'colsample_state': 71
    ,'colsample_bytree': 1.0
    ,'colsample_bylevel': 0.3
    ,'subsumple': 0.9
    ,'gamma': 0
    ,'lambda': 1
    ,'alpha': 0
    ,'min_child_weight': 1
}

num_round = 50

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dvalid = xgb.DMatrix(X_val, label=y_val)
    dtest = xgb.DMatrix(X_test)
    
    watchlist = [(dtrain, 'train'), (dvalid, 'eval')]
    model = xgb.train(params, dtrain, num_round, evals=watchlist)
    
    va_pred = model.predict(dvalid)
    score = log_loss(y_val, va_pred)
    oof_train[valid_index] = va_pred
    
    y_pred = model.predict(dtest) 
    
    
    #oof_train[valid_index] = model.predict(X_val, num_iteration=model.best_iteration)
    #y_pred = model.predict(X_test, num_iteration=model.best_iteration)##

    y_preds.append(y_pred)
    models.append(model)
    scores.append(score)

[22:00:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "colsample_state", "silent", "subsumple" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-auc:0.53876	eval-auc:0.53763
[1]	train-auc:0.53952	eval-auc:0.53624
[2]	train-auc:0.54104	eval-auc:0.53761
[3]	train-auc:0.54125	eval-auc:0.53828
[4]	train-auc:0.54111	eval-auc:0.53844
[5]	train-auc:0.54454	eval-auc:0.53930
[6]	train-auc:0.54502	eval-auc:0.53752
[7]	train-auc:0.54548	eval-auc:0.53783
[8]	train-auc:0.54688	eval-auc:0.53868
[9]	train-auc:0.54830	eval-auc:0.53817
[10]	train-auc:0.54936	eval-auc:0.53842
[11]	train-auc:0.55056	eval-auc:0.53952
[12]	train-auc:0.55239	eval-auc:0.53984
[13]	train-auc:0.55193	eval-auc:0.54064
[14

[12]	train-auc:0.55163	eval-auc:0.53762
[13]	train-auc:0.55204	eval-auc:0.53827
[14]	train-auc:0.55268	eval-auc:0.53766
[15]	train-auc:0.55253	eval-auc:0.53828
[16]	train-auc:0.55302	eval-auc:0.53848
[17]	train-auc:0.55377	eval-auc:0.53892
[18]	train-auc:0.55458	eval-auc:0.53969
[19]	train-auc:0.55468	eval-auc:0.53850
[20]	train-auc:0.55630	eval-auc:0.53877
[21]	train-auc:0.55690	eval-auc:0.53945
[22]	train-auc:0.55726	eval-auc:0.53924
[23]	train-auc:0.55808	eval-auc:0.53998
[24]	train-auc:0.55847	eval-auc:0.54016
[25]	train-auc:0.55930	eval-auc:0.54014
[26]	train-auc:0.55974	eval-auc:0.53985
[27]	train-auc:0.56006	eval-auc:0.54018
[28]	train-auc:0.56091	eval-auc:0.53910
[29]	train-auc:0.56109	eval-auc:0.53840
[30]	train-auc:0.56154	eval-auc:0.53877
[31]	train-auc:0.56211	eval-auc:0.53827
[32]	train-auc:0.56243	eval-auc:0.53874
[33]	train-auc:0.56253	eval-auc:0.53895
[34]	train-auc:0.56375	eval-auc:0.53825
[35]	train-auc:0.56370	eval-auc:0.53849
[36]	train-auc:0.56379	eval-auc:0.53849


In [126]:
y_preds

[array([0.07626409, 0.07626409, 0.07626409, ..., 0.07536193, 0.07536193,
        0.14064363], dtype=float32),
 array([0.07343386, 0.07343386, 0.07343386, ..., 0.07468738, 0.07468738,
        0.13207898], dtype=float32),
 array([0.07617155, 0.07617155, 0.07617155, ..., 0.0742854 , 0.0742854 ,
        0.22011252], dtype=float32),
 array([0.07355338, 0.07355338, 0.07355338, ..., 0.07592491, 0.07592491,
        0.14249091], dtype=float32),
 array([0.07424549, 0.07424549, 0.07424549, ..., 0.07388395, 0.07388395,
        0.1859375 ], dtype=float32)]

In [127]:
scores

[0.27468497222269306,
 0.27907147126685483,
 0.2789412900401883,
 0.27961613159677723,
 0.28098564091989303]

In [128]:
# CSV-00
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[0]
submit_all.isnull().sum()

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_00.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [129]:
# CSV-01
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[1]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_01.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [130]:
# CSV-02
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[2]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_02.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [131]:
# CSV-03
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[3]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_03.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [132]:
# CSV-04
submit_all = app_test[["SK_ID_CURR"]].copy()
# 予測確率の変数名はTARGET
submit_all["TARGET"] = y_preds[4]

# Submit File
submit_all.to_csv(
    path_or_buf="./home-credit-default-risk/submit/submit_all_04.csv", # 出力先
    sep=",",                                            # 区切り文字
    index=False,                                        # indexの出力有無
    header=True                                        # headerの出力有無
)

In [133]:
submit_all_00 = pd.read_csv('./home-credit-default-risk/submit/submit_all_00.csv')
submit_all_01 = pd.read_csv('./home-credit-default-risk/submit/submit_all_01.csv')
submit_all_02 = pd.read_csv('./home-credit-default-risk/submit/submit_all_02.csv')
submit_all_03 = pd.read_csv('./home-credit-default-risk/submit/submit_all_03.csv')
submit_all_04 = pd.read_csv('./home-credit-default-risk/submit/submit_all_04.csv')

In [134]:
submit_all = pd.DataFrame({
    'submit_all_00': submit_all_00['TARGET'].values
    ,'submit_all_01': submit_all_01['TARGET'].values
    ,'submit_all_02': submit_all_02['TARGET'].values
    ,'submit_all_03': submit_all_03['TARGET'].values
    ,'submit_all_04': submit_all_04['TARGET'].values
    })
submit_all.head()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
0,0.076264,0.073434,0.076172,0.073553,0.074245
1,0.076264,0.073434,0.076172,0.073553,0.074245
2,0.076264,0.073434,0.076172,0.073553,0.074245
3,0.108985,0.107257,0.090588,0.116346,0.069415
4,0.076264,0.073434,0.076172,0.073553,0.074245


In [135]:
submit_all.corr()

,submit_all_00,submit_all_01,submit_all_02,submit_all_03,submit_all_04
submit_all_00,1.000000,0.876941,0.878082,0.876963,0.880698
submit_all_01,0.876941,1.000000,0.874997,0.870284,0.877397
submit_all_02,0.878082,0.874997,1.000000,0.876675,0.880595
submit_all_03,0.876963,0.870284,0.876675,1.000000,0.879236
submit_all_04,0.880698,0.877397,0.880595,0.879236,1.000000


In [136]:
submit_all = app_test[["SK_ID_CURR"]].copy()
submit_all['TARGET'] = (submit_all_00['TARGET'] + submit_all_01['TARGET'] + submit_all_02['TARGET'] + submit_all_03['TARGET'] + submit_all_04['TARGET'])/5
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.074734
1,100005,0.074734
2,100013,0.074734
3,100028,0.098518
4,100038,0.074734


In [137]:
submit_all['TARGET'] = (submit_all['TARGET'] / 5)
submit_all.to_csv('./home-credit-default-risk/submit/submit_hc_only_revo_test.csv', index=False)
submit_all.head()

,SK_ID_CURR,TARGET
0,100001,0.014947
1,100005,0.014947
2,100013,0.014947
3,100028,0.019704
4,100038,0.014947


In [138]:
submit_all.count()

SK_ID_CURR    48744
TARGET        48744
dtype: int64